<a href="https://colab.research.google.com/github/MathMachado/eDreams/blob/master/eDreams_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Load Main Python libraries



In [0]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load dataframes: training & test sample

In [0]:
url_train= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/train.csv?token=AGDJQ6YO3T3X2CYDGGUPRO252KACE"
url_test= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/test.csv?token=AGDJQ623H4TWE2QKQWENGA252KAD6"

# Stacking training and validation samples for a single treatment
df_train= pd.read_csv(url_train, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])
df_test= pd.read_csv(url_test, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])

# Resetting the test sample indices
df_test.index= range(50000, 80000)

# merge train and test
df = df_train.append(df_test, sort= True)

# Records training and test dataframe indexes to separate these dataframes later
train_index = df_train.index
test_index = df_test.index

In [0]:
df.shape

In [0]:
df.head()

In [0]:
df.tail()

In [0]:
df_test.head()

In [0]:
df_test.tail()

In [0]:
df.info()

# Data Preparation

In [0]:
df_T= df.copy()
# Capturing the Company: First 2 positions of WEBSITE.
df_T['COMPANY']= df_T['WEBSITE'].str[0:2].astype(str)

# Capturing the Country: rest of the string of WEBSITE.
df_T['COUNTRY']= df_T['WEBSITE'].str[2:len(df['WEBSITE'])].astype(str)

df_T.head()

In [0]:
df_T['COMPANY'].value_counts() 

There's no 'TL'. So I'll replace 'TL' by 'MV'.

In [0]:
df_T['COMPANY']= df_T['COMPANY'].replace('TL', 'MV')
df_T['COMPANY'].value_counts() 

In [0]:
df_T['COUNTRY'].value_counts() 

In [0]:
# Corrigindo Poland Abbreviation
df_T['COUNTRY']= df_T['COUNTRY'].replace({'PLC': 'PL'})

# Corrigindo France Abbreviation
df_T['COUNTRY']= df_T['COUNTRY'].replace({'FRC': 'FR'})

# Corrigindo DEC Abbreviation
df_T['COUNTRY']= df_T['COUNTRY'].replace({'DEC': 'DE'})

# Corrigindo DEC Abbreviation
df_T['COUNTRY']= df_T['COUNTRY'].replace({'DKC': 'DK'})

df_T['COUNTRY'].value_counts() 

In [0]:
#df_T['COUNTRY']= df_T['COUNTRY'].replace(['PLC', 'DEC', 'DKC', 'FRC'], 'MV')
#df_T['COUNTRY'].value_counts() 

In [0]:
df_T['COUNTRY']= df_T['COUNTRY'].replace(['UK'], 'GB')
df_T['COUNTRY'].value_counts() 

## Treating date variables
> Since there is no information regarding the year of the transaction, I will assume that the transactions are from 2018 or 2019. I will assign the year conveniently from the analysis of the variables DEPARTURE and ARRIVAL.

In [0]:
df2= df_T.copy()
df2['DEPARTURE_WITH_YEAR']= df2['DEPARTURE'] +'/2018'
df2['ARRIVAL_WITH_YEAR']= df2['ARRIVAL'] +'/2018'
df2['ARRIVAL_WITH_YEAR_FIXED']= df2['ARRIVAL'] +'/2019'

df2['DEPARTURE_WITH_YEAR']= pd.to_datetime(df2['DEPARTURE_WITH_YEAR'])
df2['ARRIVAL_WITH_YEAR']= pd.to_datetime(df2['ARRIVAL_WITH_YEAR'])
df2['ARRIVAL_WITH_YEAR_FIXED']= pd.to_datetime(df2['ARRIVAL_WITH_YEAR_FIXED'])
df2.head()

As we do not have year information, in some cases/rows we have ARRIVAL < DEPARTURE. Let's take a look in some cases where ARRIVAL < DEPARTURE:

In [0]:
df3= df2.copy()

# I created the variable IS_ARRIVAL_BEFORE_DEPARTURE to help us identify when ARRIVAL < DEPARTURE:
df3['IS_ARRIVAL_BEFORE_DEPARTURE']= df3['ARRIVAL_WITH_YEAR'] < df3['DEPARTURE_WITH_YEAR']

# Fixing cases when ARRIVAL < DEPARTURE
df3.loc[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True, 'ARRIVAL_WITH_YEAR']= df3['ARRIVAL_WITH_YEAR_FIXED']
df3[['ARRIVAL', 'DEPARTURE', 'DEPARTURE_WITH_YEAR', 'ARRIVAL_WITH_YEAR']][df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True].head()

> Take for example line 15 (output above):
* DEPARTURE= December 15th;
* ARRIVAL= January 29th.

> Without information for the year, then ARRIVAL < DEPARTURE. However, look at the variables DEPARTURE_WITH_YEAR and ARRIVAL_WITH_YEAR above:
* DEPARTURE_WITH_YEAR= December 15th of 2018;
* ARRIVAL_WITH_YEAR= January 29th of 2019.

In this case, we fixed the problem.



In [0]:
# Drop the unnecessary variables:
df3= df3.drop(columns= ['DEPARTURE', 'ARRIVAL', 'ARRIVAL_WITH_YEAR_FIXED', 'WEBSITE'])

Next, we calculate the variable DIF_ARRIVAL_DEPARTURE = ARRIVAL_WITH_YEAR - DEPARTURE_WITH_YEAR:

In [0]:
# Calculate the variable DIF_ARRIVAL_DEPARTURE:
df3['DIF_ARRIVAL_DEPARTURE']= (df3['ARRIVAL_WITH_YEAR']-df3['DEPARTURE_WITH_YEAR']).dt.days.astype(int)

# Show some cases:
df3[['DEPARTURE_WITH_YEAR', 'ARRIVAL_WITH_YEAR', 'DIF_ARRIVAL_DEPARTURE']].head() #[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True].head()

> Something strange with the variable DIF_ARRIVAL_DEPARTURE. Take a look at the line 2 (above). We have:
* DEPARTURE_WITH_YEAR= 2018-07-29
* ARRIVAL_WITH_YEAR= 2018-08-19

It is 21 days between DEPARTURE and ARRIVAL!

Let's take a look in some statistics below. For example, let's look at the proportion of cases where DIF_ARRIVAL_DEPARTURE > 5. I am using 5 as an example, but I consider 5 a long time between departure and arrival.

In [0]:
df3_Zoom= df3[df3['DIF_ARRIVAL_DEPARTURE'] < 0]
df3_Zoom.shape[0]

In [0]:
df3_Zoom= df3[df3['DIF_ARRIVAL_DEPARTURE'] > 5]
df3_Zoom.shape[0]

In [0]:
df3_Zoom.head()

Strangely, many cases over 5 days. 

Below, I present the distribution of the DIF_ARRIVAL_DEPARTURE variable. As we can see, there are cases older than 50 days!

In [0]:
sns.distplot(df3['DIF_ARRIVAL_DEPARTURE'])

Below I present some descriptive statistics for ARRIVAL_DEPARTURE.

Let's take a look at the Boxplot:

In [0]:
plt.rcdefaults()
sns.catplot(y='DIF_ARRIVAL_DEPARTURE', kind="box", data=df3, height=4, aspect=1.5)
plt.show()

In [0]:
df3['TRIP_TYPE'].value_counts() 

Function to detect Outliers based on IQR-Score:

In [0]:
# Function that identify outlier using IQR-Score:
def IQR_Score_Outlier_Detect(df, column):
    #global df_T

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    Lim_Inf= Q1-1.5*IQR
    Lim_Sup= Q3+1.5*IQR
    print(Lim_Inf, Lim_Sup)

    # Identify the outlier
    df[column+'_OUTLIER_IQR']= np.where(((df[column] < Lim_Inf)), True, False)
    df[column+'_OUTLIER_IQR']= np.where(((df[column] > Lim_Sup)), True, False)

    # Replace outliers by Lim_Inf and Lim_Sup
    df[column+'_IQR'] = np.where(((df[column] < Lim_Inf)), Lim_Inf, df[column])
    df[column+'_IQR'] = np.where(((df[column] > Lim_Sup)), Lim_Sup, df[column])   

In [0]:
# Normalizando:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_MinMaxScaler = MinMaxScaler()
scaler_StandardScaler = StandardScaler()

In [0]:
df4= df3.copy()
df4['DIF_ARRIVAL_DEPARTURE_S']= df4['DIF_ARRIVAL_DEPARTURE']
df4['DIF_ARRIVAL_DEPARTURE_2']= df4['DIF_ARRIVAL_DEPARTURE']
df4.head()

In [0]:
df4['DIF_ARRIVAL_DEPARTURE_S']= scaler_MinMaxScaler.fit_transform(df4[['DIF_ARRIVAL_DEPARTURE_S']])
df4.head()

In [0]:
IQR_Score_Outlier_Detect(df4,'DIF_ARRIVAL_DEPARTURE_2')
df4[['DIF_ARRIVAL_DEPARTURE_2', 'DIF_ARRIVAL_DEPARTURE_2_IQR']].loc[df4['DIF_ARRIVAL_DEPARTURE_2_OUTLIER_IQR'] == True]

In [0]:
# Deleting Unneeded Variables
df5= df4.copy()
df5= df5.drop(columns= ['TIMESTAMP','DEPARTURE_WITH_YEAR','ARRIVAL_WITH_YEAR', 'IS_ARRIVAL_BEFORE_DEPARTURE'], axis= 1)

## Handling Missing Values

In [0]:
df5.info()

> Apparently we have some problems from Missing Values to DEVICE. Don't worry about the Missing values of the EXTRA_BAGGAGE variable that is our response variable and the 30,000 Missing values presented come from the test sample and are just the values we want to predict.

In [0]:
# Converting column DISTANCE to numeric. For this purpose, I'll cut the distance in the ","
df6= df5.copy()
df6[['DISTANCE_2','DISTANCE_REST']] = df6['DISTANCE'].str.split(",",expand=True)
df6['DISTANCE_2']= pd.to_numeric(df6['DISTANCE_2'])
df6[['HAUL_TYPE','DISTANCE','DISTANCE_2','DISTANCE_REST']].head(10)

Something strange with the minimum of DISTANCE_2. No sense DOMESTIC = 0. Much less INTERCONTINENTAL = 0. Let's investigate this a little further. However, I will work with DISTANCE_2 (following I will rename DISTANCE_2 TO DISTANCE) and disregard DISTANCE_REST.

In [0]:
df6= df6.drop(columns= ['DISTANCE_REST','DISTANCE'], axis= 1)
df6= df6.rename({'DISTANCE_2': 'DISTANCE'}, axis=1)
df6.head()

In [0]:
# How many cases where DISTANCE = 0?
df6[['DISTANCE']][df6['DISTANCE']==0].count()

There are 288 records where DISTANCE = 0. I consider these records to be Missing Values.

In [0]:
# Identifying Missing Values in DISTANCE. In this case, zeros.
df6.loc[df6['DISTANCE'] == 0, 'DISTANCE']= np.nan

# Checking Missing Values
df6.isna().sum()

Let's treat Missing Values in DISTANCE and DEVICE below:

In [0]:
df6['DISTANCE'] = np.where((df6['DISTANCE'].isnull()), df6['DISTANCE'].median(), df6['DISTANCE'])
df6['DISTANCE_2']= df6['DISTANCE']

In [0]:
# Checking Missing Values
df6.isna().sum()

In [0]:
# Treating Missing Values of DEVICE
df6['DEVICE'].value_counts() 

In [0]:
# Replacing NaN's of DEVICE with 'NO_DEVICE'
df6["DEVICE"].fillna("NO_DEVICE", inplace= True)

# Checking Missing Values
df6.isna().sum()

As we can see above, missing values have been addressed.

# Handling Outliers in DISTANCE using IQR-Score
> Consider the following output:

In [0]:
df7= df6.copy()
df7.head()

In [0]:
df7['DISTANCE_S']= df7['DISTANCE_2']
df7['DISTANCE_S']= scaler_MinMaxScaler.fit_transform(df7[['DISTANCE_S']])

IQR_Score_Outlier_Detect(df7, 'DISTANCE_2')
df7.head()

# Binning numeric features

In [0]:
df9= df7.copy()
df9['EXTRA_BAGGAGE'].value_counts() 

In [0]:
d_Var_Target= {True: 1, False: 0}
df9['EXTRA_BAGGAGE']= df9['EXTRA_BAGGAGE'].map(d_Var_Target)
df9.head()

In [0]:
df9.info()

In [0]:
df9.head()

## Treating categorical variables

In [0]:
df10= df9.copy()
df10.info()

In [0]:
l_Vars_Obj= list(df10.select_dtypes(include=['category', 'object']).columns)
l_Vars_Obj

In [0]:
df10.head()

In [0]:
df10 = pd.get_dummies(df10, columns= l_Vars_Obj, drop_first=True)
df10.head()

# Modeling

## Train/Test Split

### Balancing the training sample

In [0]:
df11= df10.copy()
X= df11[df11['EXTRA_BAGGAGE'].notna()]
X= X.drop(columns= ['EXTRA_BAGGAGE'], axis= 1)
y= df11[['EXTRA_BAGGAGE']][df11['EXTRA_BAGGAGE'].notna()]

In [0]:
sns.countplot(x= 'EXTRA_BAGGAGE', data= df11, palette= 'hls')

So, I'll select 10,000 EXTRA_BAGGAGE= 0 and all EXTRA_BAGGAGE= 1.

In [0]:
# Selecting all EXTRA_BAGGAGE= 1...
df11_1= df11[df11['EXTRA_BAGGAGE']== 1]
df11_1.shape

In [0]:
# Selecting all EXTRA_BAGGAGE= 1...
df11_temp= df11[df11['EXTRA_BAGGAGE']== 0]
df11_0= df11_temp.sample(n= 15000)
df11_0.shape

In [0]:
df11_2= df11_1.append(df11_0)
df11_2.shape

The following is the training sample, which we will balance.

In [0]:
df12= df11_2.copy()
X= df12[df12['EXTRA_BAGGAGE'].notna()]
X= X.drop(columns= ['EXTRA_BAGGAGE'], axis= 1)
y= df12[['EXTRA_BAGGAGE']][df12['EXTRA_BAGGAGE'].notna()]

print(X.shape, y.shape)

Importance Sampling through 'Random Forest':

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 500, max_depth=15)
rf_clf.fit(X, y)
rf_y_pred = rf_clf.predict(X)

pd.Series(rf_clf.feature_importances_, index = X.columns).nlargest(30).plot(kind = 'barh',
                                                                               figsize = (9, 9),
                                                                              title = 'Feature importance from RandomForest').invert_yaxis();

# Dropping variables

In [0]:
df13= df12.copy()
df13.head()

In [0]:
df13= df13.drop(columns= ['DISTANCE','DISTANCE_S','DISTANCE_2_IQR','DIF_ARRIVAL_DEPARTURE_2','DIF_ARRIVAL_DEPARTURE','DIF_ARRIVAL_DEPARTURE_2_IQR'])

In [0]:
X= df13[df13['EXTRA_BAGGAGE'].notna()]
X= X.drop(columns= ['EXTRA_BAGGAGE'], axis= 1)
y= df13[['EXTRA_BAGGAGE']][df13['EXTRA_BAGGAGE'].notna()]

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 500, max_depth=15)
rf_clf.fit(X, y)
rf_y_pred = rf_clf.predict(X)

pd.Series(rf_clf.feature_importances_, index = X.columns).nlargest(30).plot(kind = 'barh',
                                                                               figsize = (9, 9),
                                                                              title = 'Feature importance from RandomForest').invert_yaxis();

In [0]:
from sklearn.feature_selection import SelectFromModel

# Create a list of feature names
feat_labels = X.columns

# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.005
sfm = SelectFromModel(rf_clf, threshold=0.005)

# Train the selector
sfm.fit(X, y)

In [0]:
# Print the names of the most important features
l_Vars= []
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])
    l_Vars.append(feat_labels[feature_list_index])

Note below that we have reduced the number of columns/variables. Instead of 112, now we are working only with 28.

In [0]:
X2= df13[l_Vars][df13['EXTRA_BAGGAGE'].notna()]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2 y, test_size= 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Next, we will apply the following estimators / classifiers to the training sample:

# Evaluation
> As I'll submit a binary output, I need to use a F1-Score, as suggested in the challenge. Firstly, let's understand what's F1-Score metric:

* **Precision**: When the model predicts positive, how often is it correct? A low precision can also indicate a large number of False Positives.

    $Precision= \frac{TruePositives}{TruePositive + FalsePositives}$

* **Recall**: Recall is the number of True Positives divided by the number of True Positives and the number of False Negatives. Put another way it is the number of positive predictions divided by the number of positive class values in the test data. It is also called Sensitivity or the True Positive Rate. A low recall indicates many False Negatives.

    $Recall= \frac{TruePositives}{TruePositives + FalseNegatives}$

* **F1 Score**: F1 score conveys the balance between the precision and the recall.

    $F1= 2*\frac{Precision*Recall}{Precision+Recall}$

Source: [Classification Accuracy is Not Enough: More Performance Measures You Can Use](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/)

## Interpretation
> A good F1 score means that you have low false positives and low false negatives, so you’re correctly identifying real threats and you are not disturbed by false alarms. 
>> An F1 score is considered perfect when it’s 1, while the model is a total failure when it’s 0.

In [0]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

In [0]:
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=False,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize= (8,8),
                          cmap='Blues'):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)

### Random Forest Classifier

In [0]:
from sklearn.ensemble import RandomForestClassifier

Model_RF= RandomForestClassifier(n_estimators= 100, n_jobs= -1, min_samples_split= 2)
Model_RF.fit(X_train,y_train)
y_pred = Model_RF.predict(X_test)
accuracy_score(y_test, y_pred)

In [0]:
# Definindo a função para o GridSearchCV
def GridSearchOptimizer(modelo, Model_Opt, d_Parametros, X_train, y_train, X_test, y_test, cv= i_CV):
    Model_GridSearchCV = GridSearchCV(modelo, d_Parametros, cv= i_CV, n_jobs= -1, verbose= 10, scoring= 'accuracy')
    Model_GridSearchCV.fit(X_train, y_train)
    #print(f"\nGridSearchCV levou {tempo_elapsed:.2f} segundos.")

    # Parâmetros que otimizam a classificação:
    print(f'\nParametros otimizados: {Model_GridSearchCV.best_params_}')
    
    if Model_Opt== 'Model_DT2':
        print(f'\nDecisionTreeClassifier *********************************************************************************************************')
        Model_Opt= DecisionTreeClassifier(criterion= Model_GridSearchCV.best_params_['criterion'], 
                                          max_depth= Model_GridSearchCV.best_params_['max_depth'],
                                          max_leaf_nodes= Model_GridSearchCV.best_params_['max_leaf_nodes'],
                                          min_samples_split= Model_GridSearchCV.best_params_['min_samples_leaf'],
                                          min_samples_leaf= Model_GridSearchCV.best_params_['min_samples_split'], 
                                          random_state= i_Seed)
    elif Model_Opt== 'Model_RF2':
        print(f'\nRandomForestClassifier *********************************************************************************************************')
        Model_Opt= RandomForestClassifier(bootstrap= Model_GridSearchCV.best_params_['bootstrap'],
                                          max_depth= Model_GridSearchCV.best_params_['max_depth'],
                                          max_features= Model_GridSearchCV.best_params_['max_features'],
                                          min_samples_leaf= Model_GridSearchCV.best_params_['min_samples_leaf'],
                                          min_samples_split= Model_GridSearchCV.best_params_['min_samples_split'],
                                          n_estimators= Model_GridSearchCV.best_params_['n_estimators'],
                                          random_state= i_Seed)
        
    elif Model_Opt== 'Model_AB2':
        print(f'\nAdaBoostClassifier *********************************************************************************************************')
        Model_Opt= AdaBoostClassifier(algorithm='SAMME.R',
                                      base_estimator=RandomForestClassifier(bootstrap= False, 
                                                                            max_depth= 10,
                                                                            max_features= 'auto',
                                                                            min_samples_leaf= 1, 
                                                                            min_samples_split= 2,
                                                                            n_estimators= 400),
                                      learning_rate= Model_GridSearchCV.best_params_['learning_rate'], 
                                      n_estimators= Model_GridSearchCV.best_params_['n_estimators'], 
                                      random_state=i_Seed)
        
    elif Model_Opt== 'Model_GB2':
        print(f'\nGradientBoostingClassifier *********************************************************************************************************')
        Model_Opt= GradientBoostingClassifier(learning_rate= Model_GridSearchCV.best_params_['learning_rate'],
                                              n_estimators= Model_GridSearchCV.best_params_['n_estimators'],
                                              max_depth= Model_GridSearchCV.best_params_['max_depth'],
                                              min_samples_split= Model_GridSearchCV.best_params_['min_samples_split'],
                                              min_samples_leaf= Model_GridSearchCV.best_params_['min_samples_leaf'],
                                              max_features= Model_GridSearchCV.best_params_['max_features'])
        
    elif Model_Opt== 'Model_XGB2':
        print(f'\nXGBoostingClassifier *********************************************************************************************************')
        Model_Opt= XGBoostingClassifier(learning_rate= Model_GridSearchCV.best_params_['learning_rate'],
                                        max_depth= Model_GridSearchCV.best_params_['max_depth'],
                                        colsample_bytree= Model_GridSearchCV.best_params_['colsample_bytree'],
                                        subsample= Model_GridSearchCV.best_params_['subsample'],
                                        gamma= Model_GridSearchCV.best_params_['gamma'],
                                        min_child_weight= Model_GridSearchCV.best_params_['min_child_weight'])
        
    # Treina novamente usando os parametros otimizados...
    Model_Opt.fit(X_train, y_train)

    # Cross-Validation com 10 folds
    print(f'\n********* CROSS-VALIDATION ***********')
    a_Scores_CV = cross_val_score(Model_Opt, X_train, y_train, cv= i_CV)
    print(f'Média das Acurácias calculadas pelo CV....: {100*round(a_Scores_CV.mean(),4)}')
    print(f'std médio das Acurácias calculadas pelo CV: {100*round(a_Scores_CV.std(),4)}')

    # Faz predições com os parametros otimizados...
    y_pred = Model_Opt.predict(X_test)
  
    # Importância das COLUNAS
    print(f'\n********* IMPORTÂNCIA DAS COLUNAS ***********')
    df_Importance= pd.DataFrame(zip(l_Col_Names, Model_Opt.feature_importances_), columns= ['coluna', 'importancia'])
    df_Importance= df_Importance.sort_values(by= ['importancia'], ascending=False)
    print(df_Importance)

    # Matriz de Confusão
    print(f'\n********* CONFUSION MATRIX - PARAMETER TUNNING ***********')
    cf_matrix = confusion_matrix(y_test, y_pred)
    cf_labels = ['True Neg','False Pos','False Neg','True Pos']
    cf_categories = ['Zero', 'One']
    make_confusion_matrix(cf_matrix, group_names= cf_labels, categories= cf_categories)

    return Model_Opt, Model_GridSearchCV.best_params_

In [0]:
# Dicionário de parâmetros para o parameter tunning.
i_CV= 3
i_Seed= 20111974

d_Parametros_RF= {'criterion': ['gini', 'entropy'],
                  'max_depth': [5, 10, 15, 20, 40, 50, 100, None],
                  'min_samples_leaf': [1, 2, 4],
                  'min_samples_split': [2, 5, 10]}

In [0]:
# Invoca a função
Model_RF2, best_params= GridSearchOptimizer(Model_RF, 'Model_RF2', d_Parametros_RF, X_train, y_train, X_test, y_test, cv= i_CV)

In [0]:
# Como o procedimento acima levou 194 minutos para executar, então vou estimar Model_RF2 abaixo usando os parâmetros acima estimados
best_params= {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10}

Model_RF2= RandomForestClassifier(criterion= 'gini', 
                                  max_depth= 15, 
                                  min_samples_leaf= 1, 
                                  min_samples_split= 10, 
                                  random_state= i_Seed)

In [0]:
Model_RF2.fit(X_train,y_train)
y_pred = Model_RF2.predict(X_test)
accuracy_score(y_test, y_pred)

### Bagging Classifier

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

# Instancia...
Model_GB=GradientBoostingClassifier(n_estimators=100, min_samples_split= 2)

# Treina...
Model_GB.fit(X_train, y_train)

In [0]:
# Cross-Validation com 10 folds
a_Scores_CV = cross_val_score(Model_GB, X_train, y_train, cv= i_CV)
print(f'Média das Acurácias calculadas pelo CV....: {100*round(a_Scores_CV.mean(),4)}')
print(f'std médio das Acurácias calculadas pelo CV: {100*round(a_Scores_CV.std(),4)}')

In [0]:
# Faz precições...
y_pred = Model_GB.predict(X_test)

# Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_pred)
cf_labels = ['True Neg','False Pos','False Neg','True Pos']
cf_categories = ['Zero', 'One']
make_confusion_matrix(cf_matrix, group_names= cf_labels, categories= cf_categories)

In [0]:
# Dicionário de parâmetros para o parameter tunning.
d_Parametros_GB= {'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
                  'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
                  'max_depth': [5, 10, 15, 20, 25, 30],
                  'min_samples_split': [0.1, 0.3, 0.5, 0.7, 0.9],
                  'min_samples_leaf': [0.1, 0.2, 0.3, 0.4, 0.5],
                  'max_features': list(range(1, X_train.shape[1]))}

In [0]:
# Invoca a função
Model_GB2, best_params= GridSearchOptimizer(Model_GB, 'Model_GB2', d_Parametros_GB, X_train, y_train, X_test, y_test, cv= i_CV)

In [0]:
# Como o procedimento acima levou 93 minutos para executar, então vou estimar Model_GB2 abaixo usando os parâmetros acima estimados
best_params= {'learning_rate': 1, 'max_depth': 30, 'max_features': 11, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 100}

In [0]:
Model_GB2= GradientBoostingClassifier(learning_rate= best_params['learning_rate'], 
                                      max_depth= best_params['max_depth'],
                                      min_samples_leaf= best_params['min_samples_leaf'],
                                      min_samples_split= best_params['min_samples_split'],
                                      n_estimators= best_params['n_estimators'],
                                      random_state= i_Seed)

# Submitting my model

In [0]:
X_Val= df10[l_Vars][df10['EXTRA_BAGGAGE'].isna()]
y_Val= df10[['EXTRA_BAGGAGE']][df10['EXTRA_BAGGAGE'].isna()]

print(X_Val.shape, y_Val.shape)

In [0]:
X_Val['EXTRA_BAGGAGE'] = clf.predict(X_Val)
y_pred_submission= X_Val[['EXTRA_BAGGAGE']]
y_pred_submission.head()

In [0]:
y_pred_submission['EXTRA_BAGGAGE']= y_pred_submission['EXTRA_BAGGAGE'].map({0.0: False, 1.0: True})
y_pred_submission.head()

In [0]:
y_pred_submission.to_csv(r'eDreams_Submission.csv')